In [ ]:
# %load sentiment_stream.py
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API 
access_token = "925147187184635904-DdhQn8xZKPVFwHb68mYUHuhzpi8Anzu"
access_token_secret = "8I4Lg5fz5sNPawKccZXpjQCu4azXEa3UbFNjvyAe2xTWY"
consumer_key = "KT6cdia0sVGwrmyqX0m4xXbac"
consumer_secret = "ovXZkyp7k4jih8LalVRVHfLCxZTn9TztolHmMCvbBtOyDLLVYw"


#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):

    def on_data(self, data):
        print data
        return True

    def on_error(self, status):
        print status


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'football', 'eagles', 'cowboys', 'packers' , 'giants'.
    stream.filter(track=['selfdrive', 'automatic', 'tesla', 'autonomous'])

In [ ]:
%%cmd
python sentiment_stream.py>sentiment.txt

In [ ]:
import json
import pandas as pd
tweets_data_path = 'sentiment.txt'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
        
print len(tweets_data)

tweets = pd.DataFrame()
tweets['text'] = map(lambda tweet: tweet['text'], tweets_data)
tweets['lang'] = map(lambda tweet: tweet['lang'], tweets_data)
tweets['time_zone'] = map(lambda tweet: tweet['user']['time_zone'], tweets_data)
tweets_by_lang = tweets['lang'].value_counts()
tweets

In [ ]:
import json, nltk, re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean_data(tweets_data_path):
    tweets_data = []
    stop_words = set(stopwords.words('english'))
    tweets_file = open(tweets_data_path, "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            if tweet["lang"]=='en':
                tweet_text = tweet["text"]
                #print tweet_text
                filter_url = re.sub(r"http\S+", "", tweet_text)
                #print filter_url
                filter_names = re.sub(r"RT+.*\:", "", filter_url)
                #print filter_names
                filter_splchar = re.sub("[^A-Z a-z]+", "", filter_names)
                #print filter_splchar
                filter_stop = [word for word in (filter_splchar.lower()).split() if word not in stop_words]
                result = ' '.join(filter_stop)
                #print result
                tweets_data.extend(result.split())       
        except:
            continue
            
    return tweets_data

In [ ]:
data_path = 'sentiment.txt'
tweet_data = clean_data(data_path)
words = set(tweet_data)
file = open('sentiment_words.txt', 'w')
for item in list(words):
    file.write("%s\n" % item)

In [ ]:
top_train_words=pd.DataFrame(tweet_data)
pd.DataFrame(top_train_words[0].value_counts()).head(10)

In [ ]:
from pybloom import BloomFilter
import os
import re

post_dir = 'C:\\Users\\srira\\Desktop\\positive\\'
# Read all my posts.
posts = {post_name: open(post_dir + post_name).read() for post_name in os.listdir(post_dir)}
# Create a dictionary of {"post name": "lowercase word set"}.
split_posts = {name: set(re.split("\W+", contents)) for name, contents in posts.items()}

In [ ]:
from pybloom import BloomFilter
import os
import re

post_dir = 'C:\\Users\\srira\\Desktop\\negative\\'
# Read all my posts.
posts = {post_name: open(post_dir + post_name).read() for post_name in os.listdir(post_dir)}
# Create a dictionary of {"post name": "lowercase word set"}.
split_posts = {name: set(re.split("\W+", contents)) for name, contents in posts.items()}

In [ ]:
filters = {}
for name, words in split_posts.items():
    print name, words
    filters[name] = BloomFilter(capacity=len(words), error_rate=0.1)
    for word in words:
        filters[name].add(word)

In [ ]:
def search(search_string):
    search_terms = re.split("\W+", search_string)
    return [name for name, filter in filters.items() if all(term in filter for term in search_terms)]

In [ ]:
import os
import re
matched_words = []
path = 'C:\\Users\\srira\\Desktop\\sentiment\\'
post = {post_name: open(path + post_name).read() for post_name in os.listdir(path)}
# Create a dictionary of {"post name": "lowercase word set"}.
split_pst = {name: set(re.split("\W+", contents.lower())) for name, contents in post.items()}
#split_posts
for name, words in split_pst.items():
    for word in words:
        if search(word)==['positive.txt']:
            matchedpositive_words.append(word)

In [ ]:
import os
import re
matched_words = []
path = 'C:\\Users\\srira\\Desktop\\sentiment\\'
post = {post_name: open(path + post_name).read() for post_name in os.listdir(path)}
# Create a dictionary of {"post name": "lowercase word set"}.
split_pst = {name: set(re.split("\W+", contents.lower())) for name, contents in post.items()}
#split_posts
for name, words in split_pst.items():
    for word in words:
        if search(word)==['negative.txt']:
            matchednegative_words.append(word)

In [ ]:
positive_match_words = [x for x in sentiment_words if x in matchedpositive_words]
sort_match_positive = sorted(positive_match_words)

In [ ]:
negative_match_words = [x for x in sentiment_words if x in matchednegative_words]
sort_match_negative = sorted(neative_match_words)

In [ ]:
a = pd.DataFrame(sort_match_positive)
matchpos=a[0].value_counts().head(10)
match

In [ ]:
b = pd.DataFrame(sort_match_negative)
matchneg=b[0].value_counts().head(10)
match

In [ ]:
import matplotlib.pyplot as plt
ax = matchpos.plot(x=0, y=1, kind="bar", figsize=(10, 10), fontsize=15)
ax.set_title('Matched positive Words Frequency', fontsize=15)
ax.set_ylabel("Frequency", fontsize=10)
ax.set_xlabel("Words", fontsize=10)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
ax = matchneg.plot(x=0, y=1, kind="bar", figsize=(10, 10), fontsize=15)
ax.set_title('Matched negative Words Frequency', fontsize=15)
ax.set_ylabel("Frequency", fontsize=10)
ax.set_xlabel("Words", fontsize=10)
plt.show()